In [85]:
import pandas as pd
import numpy as np
import pickle
from keras.models import Model, Sequential
from keras.applications import resnet50, vgg16, densenet
from keras.preprocessing import image
from sklearn.model_selection import train_test_split, KFold
from keras import backend as K
from keras.layers import *
from keras import layers
import keras
from keras.callbacks import EarlyStopping, Callback
# from skimage.io import imread
import sklearn
import lightgbm as lgb
K.set_image_data_format('channels_last')

path = '../Data/'

# class_emb = pd.read_csv(path + '/DatasetA_train_20180813/class_wordembeddings.txt', 
#                         index_col = 0, sep = ' ', header = None)
# class_emb.index.name = 'class_name'
# # class_emb_vec = pd.DataFrame(index = class_emb.index)
# class_emb = class_emb.apply(lambda s: np.array([float(x) for x in s]), axis = 1)

# class_id_to_name = pd.read_csv(path + '/DatasetA_train_20180813/label_list.txt', 
#                                index_col = 'class_name', sep = '\t', header = None, names = ['class_id', 'class_name'])

# attr_list = pd.read_csv(path + '/DatasetA_train_20180813/attribute_list.txt', index_col = 0, sep = '\t', header = None)

# attributes_per_class = pd.read_csv(path + '/DatasetA_train_20180813/attributes_per_class.txt', 
#                                    index_col = 0, sep = '\t', header = None)
# attributes_per_class.index.name = 'class_id'
# attributes_per_class = attributes_per_class.apply(lambda s: np.array([float(x) for x in s]), axis = 1)

# class_id_emb_attr = class_id_to_name.copy()
# class_id_emb_attr['emb'] = class_emb
# class_id_emb_attr.reset_index(inplace = True)
# class_id_emb_attr.set_index('class_id', inplace = True)
# class_id_emb_attr['attr'] = attributes_per_class

# with open(path + 'class_id_emb_attr.pickle', 'wb+') as handle:
#     pickle.dump(class_id_emb_attr, handle)

In [4]:
# train_data = pd.read_csv(path + '/DatasetA_train_20180813/train.txt', index_col = 'class_id', 
#                          sep = '\t', header = None, names = ['image_id', 'class_id'])
# train_data = train_data.merge(class_id_emb_attr, how = 'left', on = 'class_id')

# imag_path = path + r'/DatasetA_train_20180813/train/'
# def read_image(image_id):
#     img = image.load_img(imag_path + image_id)
#     img = image.img_to_array(img)
# #     img= vgg16.preprocess_input(image.img_to_array(img))
#     return img
# train_data['img'] = train_data['image_id'].apply(lambda id: read_image(id))

# train_data.reset_index(inplace = True)
with open(path + 'class_id_emb_attr.pickle', 'rb') as handle:
    class_id_emb_attr = pickle.load(handle)
    
with open(path + 'train_img.pickle', 'rb') as handle:
    train_data = pickle.load(handle)

In [5]:
train_data.head()

,class_id,image_id,class_name,emb,attr,img
0,ZJL1,a6394b0f513290f4651cc46792e5ac86.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[20.0, 33.0, 7.0], [19.0, 32.0, 6.0], [22.0,..."
1,ZJL1,2fb89ef2ace869d3eb3bdd3afe184e1c.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[140.0, 45.0, 51.0], [144.0, 47.0, 54.0], [1..."
2,ZJL1,eda9f3bef2bd8da038f6acbc8355fc25.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[81.0, 69.0, 21.0], [86.0, 74.0, 26.0], [85...."
3,ZJL1,7d93ef45972154aae150b4f9980a79c0.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[16.0, 14.0, 15.0], [18.0, 16.0, 17.0], [19...."
4,ZJL1,fb901b4f9a8e396c1d0155bccc5e5671.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[120.0, 124.0, 127.0], [89.0, 93.0, 96.0], [..."


In [49]:
imread(path + '/DatasetA_train_20180813/train/a6394b0f513290f4651cc46792e5ac86.jpeg').shape

(64, 64, 3)

In [7]:
category = train_data['class_id'].unique()
category_dict = dict((category[i], i) for i in range(category.shape[0]))

# vgg_model = vgg16.VGG16(include_top=True, weights = None, input_shape=(64, 64, 3), classes = train_data['class_id'].unique().shape[0])


# with open(path + 'train_vgg16_img_div255.pickle', 'rb') as handle:
#     train_y = pickle.load(handle)

# input = Input(shape = (2048, ))
# dense = Dense(2048, activation="relu")(input)
# dense = Dropout(0.5)(dense)
# dense = Dense(2048, activation="relu")(input)
# dense = Dense(1024, activation="relu")(input)
# dense = Dense(category.shape[0], activation="softmax")(input)

# vgg_model = Model(input, dense)

# vgg_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

# print (vgg_model.summary())

# train_image_feature_map = np.reshape(vgg_model.predict(train_data_part_img, verbose=  1), (train_data_part_length, -1))
# with open(path + 'train_vgg16_img_div255.pickle', 'wb+') as handle:
#     pickle.dump(train_y, handle)
# with open(path + 'train_vgg16_img_div255.pickle', 'rb') as handle:
#     train_image_feature_map = pickle.load(handle)
# train_data['target'] = list(train_image_feature_map)

def small_vgg(img_input_shape, classes):
    # Block 1
    img_input = Input(shape = (img_input_shape))
    x = layers.Conv2D(16, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv1')(img_input)
    x = layers.Conv2D(16, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

#     # Block 2
    x = layers.Conv2D(32, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv1')(x)
    x = layers.Conv2D(32, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    
#     # Block 3
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv1')(x)
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv2')(x)
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    
#     # Block 4
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv1')(x)
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv2')(x)
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
    
#     # Block 5
#     x = layers.Conv2D(256, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block5_conv1')(x)
#     x = layers.Conv2D(256, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block5_conv2')(x)
#     x = layers.Conv2D(256, (3, 3),
#                       activation='relu',
#                       padding='same',
#                       name='block5_conv3')(x)
#     x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    # Classification block
    x = layers.Flatten(name='flatten')(x)
#     x = layers.Dense(1024, activation='relu', name='fc1')(x)
#     x = layers.Dense(512, activation='relu', name='fc2')(x)
    x = layers.Dense(classes, activation='softmax', name='predictions')(x)
    
    model = Model(img_input, x)
    print (model.summary())
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
    
    return model

def small_densenet(img_input_shape, classes, blocks = [6, 12, 24, 16]):
    img_input = Input(shape = (img_input_shape))
    x = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
    x = layers.Conv2D(64, 7, strides=2, use_bias=False, name='conv1/conv')(x)
    x = layers.BatchNormalization(
        axis=3, epsilon=1.001e-5, name='conv1/bn')(x)
    x = layers.Activation('relu', name='conv1/relu')(x)
    x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = layers.MaxPooling2D(3, strides=2, name='pool1')(x)
    
    x = densenet.dense_block(x, blocks[0], name='conv2')
    x = densenet.transition_block(x, 0.5, name='pool2')
    x = densenet.dense_block(x, blocks[1], name='conv3')
    x = densenet.transition_block(x, 0.5, name='pool3')
    x = densenet.dense_block(x, blocks[2], name='conv4')
    x = densenet.transition_block(x, 0.5, name='pool4')
    x = densenet.dense_block(x, blocks[3], name='conv5')
    
    x = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name='bn')(x)
    x = layers.Activation('relu', name='relu')(x)
    
    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    x = layers.Dense(classes, activation='softmax', name='fc')(x)
    
    model = Model(img_input, x)
    print (model.summary())
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
    
    return model

train_img = extract_array_from_series(train_data['img'])
train_img = vgg16.preprocess_input(train_img)
OneHotEncoder = sklearn.preprocessing.OneHotEncoder()
train_target = train_data['class_id'].apply(lambda id: category_dict[id]).values
train_target = OneHotEncoder.fit_transform(np.reshape(train_target, (-1, 1))).toarray()

In [9]:
fold = 5
kf = KFold(n_splits=fold, shuffle=True)
num_fold = 0

# train_image_feature_map = np.reshape(vgg_model.predict(train_data_part_img, verbose=  1), (train_data_part_length, -1))
# with open(path + 'train_vgg16_img_div255.pickle', 'wb+') as handle:
#     pickle.dump(train_y, handle)
# with open(path + 'train_vgg16_img_div255.pickle', 'rb') as handle:
#     train_image_feature_map = pickle.load(handle)

for train_index, test_index in kf.split(train_img):
    train_part_img = train_img[train_index]
    validate_part_img = train_img[test_index]
    
    train_part_target = train_target[train_index]
    validate_part_target = train_target[test_index]
    
    print ('Train target nunique: ', np.unique(np.argwhere(train_part_target == 1)[:, 1]).shape[0], 
           'Validate target nuique: ', np.unique(np.argwhere(validate_part_target == 1)[:, 1]).shape[0])
#     model = vgg16.VGG16(include_top=True, weights = None, input_shape=(64, 64, 3), classes = category.shape[0])
#     model = resnet50.ResNet50(include_top=True, weights = None, input_shape=(224, 224, 3), classes = category.shape[0])
#     print (model.summary())
#     model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
    img_model = small_densenet(blocks = [2, 2, 2, 2], img_input_shape=(64, 64, 3), classes = train_target.shape[1]) 
#     print (img_model.summary())
    # img_model = small_vgg(img_input_shape = (64, 64, 3), classes = train_target.shape[1])
    h = img_model.fit(train_part_img, train_part_target, validation_data=(validate_part_img, validate_part_target), 
                  epochs=10, batch_size = 64, shuffle=True, verbose = 1, 
                  callbacks=[EarlyStopping(monitor='val_loss', patience=3, verbose=0)])
    break

Train target nunique:  190 Validate target nuique:  190
Train on 30576 samples, validate on 7645 samples
Epoch 1/10
30576/30576 [==============================] - 646s 21ms/step - loss: 3.7635 - categorical_accuracy: 0.1645 - val_loss: 3.9950 - val_categorical_accuracy: 0.1440
Epoch 2/10
30576/30576 [==============================] - 656s 21ms/step - loss: 3.5163 - categorical_accuracy: 0.2014 - val_loss: 3.5633 - val_categorical_accuracy: 0.1996
Epoch 3/10
30576/30576 [==============================] - 658s 22ms/step - loss: 3.3222 - categorical_accuracy: 0.2366 - val_loss: 3.5310 - val_categorical_accuracy: 0.2114
Epoch 4/10
30576/30576 [==============================] - 652s 21ms/step - loss: 3.1632 - categorical_accuracy: 0.2608 - val_loss: 3.5205 - val_categorical_accuracy: 0.2126
Epoch 5/10
30576/30576 [==============================] - 653s 21ms/step - loss: 3.0152 - categorical_accuracy: 0.2864 - val_loss: 3.4801 - val_categorical_accuracy: 0.2275
Epoch 6/10
30576/30576 [======

In [50]:
# df = pd.DataFrame(h.history)
# df = df.append(df)
# df.groupby(df.index).agg({'mean', 'max', 'median'})
# h.epoch
# x = pd.DataFrame()
# x.append(df)
train_img.shape

(38221, 64, 64, 3)

In [186]:
# np.asarray(list(train_data['img'])).shape
# train_data[train_data['class_id'] == 'ZJL1']
# train_data['class_id'].value_counts().hist()
# train_data.head()
# class_id_emb_attr.iloc[0].name
# resnet50_model = ResNet50(weights='imagenet', input_shape=(224, 224, 3))
img = image.load_img(path + '/DatasetA_train_20180813/train/000c0d617f5b67d116dee15c40d1d47d.jpeg', target_size=(224, 224))
img = image.img_to_array(img)
img = vgg16.preprocess_input(img)
preds = vgg_model.predict(np.expand_dims(img, axis=0))
print('Predicted:', vgg16.decode_predictions(preds, top=3)[0])
# imread(path + '/DatasetA_train_20180813/train/000c0d617f5b67d116dee15c40d1d47d.jpeg')
# image.img_to_array(img).shape

Predicted: [('n03598930', 'jigsaw_puzzle', 0.023861792), ('n02095314', 'wire-haired_fox_terrier', 0.022931756), ('n04589890', 'window_screen', 0.021361042)]


In [87]:
def full_connect_layer(input, hidden_dim):
    full_connect = input
    for hn in hidden_dim:
        fc_in = full_connect
        # if self.full_connect_dropout > 0:
        #     full_connect = Dropout(self.full_connect_dropout)(full_connect)
        # full_connect = BatchNormalization()(full_connect)
        # full_connect = Dense(hn, kernel_regularizer = l2(0.001), activity_regularizer = l1(0.001))(full_connect)
        full_connect = Concatenate()([Dense(hn, kernel_initializer='lecun_uniform', activation = 'relu')(full_connect), 
            Dense(hn, kernel_initializer='lecun_uniform', activation = 'sigmoid')(full_connect)])
        # full_connect = Dense(hn, kernel_initializer='lecun_uniform', activation = 'sigmoid')(full_connect)
        # full_connect = self.act_blend(full_connect)
        # if self.full_connect_dropout > 0:
        #     full_connect = Dropout(self.full_connect_dropout)(full_connect) #Dropout(self.full_connect_dropout)(full_connect)
        full_connect = Concatenate()([fc_in, full_connect])
    return full_connect

def create_dnn():
    alpha = 0.03
    attr_input = Input(shape = (30,))
    word_emb = Input(shape = (300,))
    imag_classifier = Input(shape = (1024,))

    attr_dense = Dense(64, use_bias = False)(attr_input)
#     attr_dense = Dense(512)(attr_dense)
#     attr_dense = LeakyReLU(alpha=alpha)(attr_dense)
#     attr_dense = Dense(256)(attr_dense)
#     attr_dense = LeakyReLU(alpha=alpha)(attr_dense)
#     attr_dense = Dense(300, activation="relu")(attr_dense)
    # attr_dense = Dense(512, activation="relu")(attr_dense)

    attr_word_emb = Concatenate()([word_emb, attr_dense])
    attr_word_emb_dense = full_connect_layer(attr_word_emb, hidden_dim = [1024, 512])
    attr_word_emb_dense = Dense(128, activation="tanh")(attr_word_emb_dense)
#     attr_word_emb_dense = attr_word_emb
#     attr_word_emb_dense = Dense(4096, activation="relu")(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(1024)(attr_word_emb)
#     attr_word_emb_dense = LeakyReLU(alpha=alpha)(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(512)(attr_word_emb_dense)
#     attr_word_emb_dense = LeakyReLU(alpha=alpha)(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(256)(attr_word_emb_dense)
#     attr_word_emb_dense = LeakyReLU(alpha=alpha)(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(128)(attr_word_emb_dense)
#     attr_word_emb_dense = LeakyReLU(alpha=alpha)(attr_word_emb_dense)
#     attr_word_emb_dense = Dropout(0.2)(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(2048, activation="relu")(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(1024, activation="relu")(attr_word_emb_dense)
#     attr_word_emb_dense = Dropout(0.2)(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(512, activation="relu")(attr_word_emb_dense)
#     attr_word_emb_dense = Dropout(0.2)(attr_word_emb_dense)
    # attr_word_emb_dense = Dense(2048, activation="sigmoid", kernel_regularizer=keras.regularizers.l2(0.01))(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(1024, activation="relu")(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(2048, activation="relu")(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(4096, activation="relu")(attr_word_emb_dense)
#     attr_word_emb_dense = Dense(8192, activation="relu")(attr_word_emb_dense)

    # mse = Dense(6, activation="softmax")(dense_output)
#     vgg_input = vgg_model.input
#     vgg_output = Flatten()(vgg_model.output)
#     vgg_output = Dense(1024, activation="sigmoid")(vgg_output)

    # mse_loss = keras.losses.mean_squared_error(vgg_output, attr_word_emb_dense)

    model = Model([attr_input, word_emb], outputs = attr_word_emb_dense) #, vgg_output])
    # model.add_loss(mse_loss)
    model.compile(optimizer='adam', loss='mean_squared_error')
#     model.summary()
    return model

def find_nearest_class(class_id_emb_attr, model, eval_df, img_feature_map = None):
    if img_feature_map is None:
        img_feature_map = np.reshape(vgg_model.predict(np.asarray(list(eval_df['img'])), verbose = 1), (eval_img.shape[0], -1))
    cand_feature_map = model.predict(create_dnn_data(class_id_emb_attr))
    # img_feature_map = np.repeat(img_feature_map, [cand_feature_map.shape[0]] * img_feature_map.shape[0], axis = 0)
    nearest_class_id = ['ZJL'] * eval_df.shape[0]
    for i in range(img_feature_map.shape[0]):
        img = img_feature_map[i]
        dis = np.linalg.norm(img - cand_feature_map, axis = 1)
        min_ind = np.where(dis == np.amin(dis))[0]
        if len(min_ind) > 1:
            print ('eval img id: ', eval_df.iloc[i]['class_id'], 'has multiple best candidates: ', len(min_ind), 'min val: ', np.amin(dis))
#         print (i, img, class_id_emb_attr.iloc[min_ind[0]].name)
        nearest_class_id[i] = class_id_emb_attr.iloc[min_ind[0]].name
#     print (nearest_class_id)
    return np.asarray(nearest_class_id)
    
    
def extract_array_from_series(s):
    return np.asarray(list(s))
    
def create_dnn_data(df):
    return [extract_array_from_series(df['attr']), extract_array_from_series(df['emb'])]
    
class AccuracyEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1, batch_interval = 1000000, verbose = 2, \
            scores = [], class_id_emb_attr = None, eval_df = None):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data
        # print("y_val shape:{0}".format(self.y_val.shape))
        self.batch_interval = batch_interval
        self.verbose = verbose
        self.scores = scores
        self.class_id_emb_attr = class_id_emb_attr
        self.eval_df = eval_df
#         self.img_feature_map = self.y_val

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            pred_nearest_class_id = find_nearest_class(self.class_id_emb_attr, self.model, self.eval_df, self.y_val)
            true_class_id = self.eval_df['class_id'].values
#             print (pred_nearest_class_id, "\n", true_class_id)
            right_num = np.sum(pred_nearest_class_id == true_class_id)
            score = right_num / true_class_id.shape[0]
            self.scores.append("epoch:{0} {1}".format(epoch + 1, score))
            print("\n Accuracy epoch: %d - score: %.6f \n %d/%d" % (epoch+1, score, right_num, true_class_id.shape[0]))
    
    def on_batch_end(self, batch, logs={}):
        return
        if(self.verbose >= 2) and (batch % self.batch_interval == 0):
            # y_pred = self.model.predict(self.X_val, verbose=0)
            # loss = metrics.log_loss(self.y_val, y_pred)
            print("Hi! on_batch_end() , batch=",batch,",logs:",logs)
            # print("Valide size=",y_pred.shape[0], "  Valide loss=",loss)
            
def lgbm_train(train_part, train_part_label, valide_part, valide_part_label, fold_seed,
        fold = 5, train_weight = None, valide_weight = None, flags = None):
    """
    LGBM Training
    """
    print("-----LGBM training-----")

    d_train = lgb.Dataset(train_part, train_part_label) #, categorical_feature = CATEGORY_FEATURES) #, init_score = train_part[:, -1])
    d_valide = lgb.Dataset(valide_part, valide_part_label) #, categorical_feature = CATEGORY_FEATURES) #, init_score = valide_part[:, -1])
    params = {
            'task': 'train',
            'boosting_type': 'gbdt', #'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            # 'num_leaves':-1,
            'min_sum_hessian_in_leaf':None,
            'max_depth':7,
            'learning_rate':0.005,
            'feature_fraction':0.1,
            'verbose':-1,
            'num_boost_round':3000,
            'drop_rate':None,
            'bagging_fraction':0.6,
            'bagging_freq':5,
            'early_stopping_round':100,
            # 'min_data_in_leaf':100,
            'max_bin': None,
            'scale_pos_weight':None,
        }
    print ("lightgbm params: {0}\n".format(params))

    bst = lgb.train(
                    params ,
                    d_train,
                    verbose_eval = 200,
                    valid_sets = [d_train, d_valide],
                    # feature_name= keras_train.DENSE_FEATURE_LIST,
                    #feval = gini_lgbm
                    #num_boost_round = 1
                    )
    #pred = model_eval(bst, 'l', valide_part)
    #print(pred[:10])
    #print(valide_part_label[:10])
    #print(valide_part[:10, -1])
    # exit(0)
    feature_imp = bst.feature_importance(importance_type = 'gain')
    sort_ind = np.argsort(feature_imp)[::-1]
    print (np.c_[np.array(FEATURE_LIST)[sort_ind], feature_imp[sort_ind]][:10])
    # print (np.array(keras_train.FEATURE_LIST)[np.argsort(feature_imp)])
    # exit(0)
    # cv_result = lgb.cv(params, d_train, nfold=fold) #, feval = gini_lgbm)
    # pd.DataFrame(cv_result).to_csv('cv_result', index = False)
    # exit(0)
    return bst

In [6]:
# with open(path + 'train_vgg16_img_div255.pickle', 'rb') as handle:
#     train_y = pickle.load(handle)
# img_model_flat = Model(input = img_model.input, output = img_model.get_layer(name = 'avg_pool').output)
# train_data['target'] = list(train_y) #
# train_data['target'] = list(img_model_flat.predict(train_img, verbose = 1))
img_model_flat = pd.read_csv(path + 'model_sub/2_2_2_2_NoFinalNormRelu/sub_2018_09_07_11_26_51.csv')

In [27]:
# img_model_flat.set_index('image_id', inplace = True)
train_data['target'] = img_model_flat.apply(lambda s: np.asarray(s.astype(float)), axis = 1).values

In [34]:
img_model_flat.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
image_id,,,,,,,,,,,,,,,,,,,,,
a6394b0f513290f4651cc46792e5ac86.jpeg,-0.111403,0.033743,-0.524962,0.311080,0.220825,-0.269303,-0.114321,-0.304165,-0.224817,-0.427597,...,-0.080115,0.562587,0.634381,0.582980,-0.908676,2.174307,-0.131301,-0.996003,-1.039952,-1.069237
2fb89ef2ace869d3eb3bdd3afe184e1c.jpeg,-0.158180,0.216972,-0.270589,0.100630,0.386496,0.356503,0.100902,-0.198551,-0.474728,-0.197594,...,0.224400,-0.526723,0.653620,-0.276354,-0.797931,1.843194,0.309933,-0.723460,-0.784283,-0.090235
eda9f3bef2bd8da038f6acbc8355fc25.jpeg,0.142461,-0.100540,0.390588,0.405993,0.145872,-0.120031,-0.424403,0.217373,0.802554,-0.434137,...,-0.784629,-0.032743,-0.022950,-0.905833,0.320548,0.529700,0.172870,-0.737320,0.896569,0.736241
7d93ef45972154aae150b4f9980a79c0.jpeg,0.030461,-0.014245,0.041093,-0.091906,0.370515,0.066534,-0.229778,-0.047806,-0.245216,-0.037742,...,-0.234935,-0.111720,0.340227,-0.784400,-1.211687,0.995485,-0.745386,0.620005,0.020696,-0.693240
fb901b4f9a8e396c1d0155bccc5e5671.jpeg,0.505966,-0.445995,0.017522,0.519428,-0.320725,-0.811842,-0.050119,-0.604761,0.432402,0.054279,...,-0.153951,-0.309546,1.746772,-0.817441,-0.643897,2.104230,-0.379953,0.387951,-0.303779,0.064668


In [28]:
train_data.head()

,class_id,image_id,class_name,emb,attr,img,target
0,ZJL1,a6394b0f513290f4651cc46792e5ac86.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[20.0, 33.0, 7.0], [19.0, 32.0, 6.0], [22.0,...","[-0.11140289, 0.03374281, -0.52496225, 0.31107..."
1,ZJL1,2fb89ef2ace869d3eb3bdd3afe184e1c.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[140.0, 45.0, 51.0], [144.0, 47.0, 54.0], [1...","[-0.15818031, 0.21697168, -0.27058864, 0.10062..."
2,ZJL1,eda9f3bef2bd8da038f6acbc8355fc25.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[81.0, 69.0, 21.0], [86.0, 74.0, 26.0], [85....","[0.14246145, -0.100540064, 0.39058790000000004..."
3,ZJL1,7d93ef45972154aae150b4f9980a79c0.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[16.0, 14.0, 15.0], [18.0, 16.0, 17.0], [19....","[0.030461132999999998, -0.014244867, 0.0410932..."
4,ZJL1,fb901b4f9a8e396c1d0155bccc5e5671.jpeg,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ...","[[[120.0, 124.0, 127.0], [89.0, 93.0, 96.0], [...","[0.50596607, -0.44599533, 0.017522344, 0.51942..."


In [89]:
#data.loc[class_id] #data_atten.loc[n]
fold = 5
kf = KFold(n_splits=fold, shuffle=True)
num_fold = 0
class_ids = train_data['class_id'].unique() #[:5]

# train_image_feature_map = np.reshape(vgg_model.predict(train_data_part_img, verbose=  1), (train_data_part_length, -1))
# with open(path + 'train_vgg16_img_div255.pickle', 'wb+') as handle:
#     pickle.dump(train_y, handle)
# with open(path + 'train_vgg16_img_div255.pickle', 'rb') as handle:
#     train_image_feature_map = pickle.load(handle)

for train_index, test_index in kf.split(class_ids):
    train_part_id = class_ids[train_index]
    validate_part_id = class_ids[test_index]
    
    train_part_df = train_data[train_data['class_id'].isin(train_part_id)]
    validate_part_df = train_data[train_data['class_id'].isin(validate_part_id)]

    train_part_data = create_dnn_data(train_part_df)
    train_part_target = extract_array_from_series(train_part_df['target'])
    
    validate_part_data = create_dnn_data(validate_part_df)
    validate_part_target = extract_array_from_series(validate_part_df['target'])
    
    print('fold: %d th train :-)' % (num_fold))
    print('Train size: {} Valide size: {}'.format(train_part_df.shape[0], validate_part_df.shape[0]))
    print('Train category: {} Valide category: {}'.format(train_part_id.shape[0], validate_part_id.shape[0]))

#     callbacks = [
#             EarlyStopping(monitor='val_loss', patience=30, verbose=0),
#             AccuracyEvaluation(validation_data=(validate_part_data, validate_part_target), interval=1, \
#                 class_id_emb_attr = class_id_emb_attr.loc[validate_part_id], eval_df = validate_part_df)
#             ]
    
#     model = create_dnn()
#     if num_fold == 0:
#         print (model.summary())
#     model.fit(train_part_data, train_part_target,  validation_data=(validate_part_data, validate_part_target),
#                   epochs=100, batch_size = 64, shuffle=True, verbose = 1, callbacks=callbacks)
    
    model = lgbm_train(train_part_data[0], train_part_target, validate_part_data[0], validate_part_target, num_fold, fold)
    num_fold += 1

fold: 0 th train :-)
Train size: 30566 Valide size: 7655
Train category: 152 Valide category: 38
-----LGBM training-----
lightgbm params: {'bagging_fraction': 0.6, 'min_sum_hessian_in_leaf': None, 'metric': 'rmse', 'task': 'train', 'feature_fraction': 0.1, 'scale_pos_weight': None, 'boosting_type': 'gbdt', 'bagging_freq': 5, 'max_bin': None, 'num_boost_round': 3000, 'learning_rate': 0.005, 'max_depth': 7, 'objective': 'regression', 'early_stopping_round': 100, 'drop_rate': None, 'verbose': -1}



/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))


TypeError: Wrong type(ndarray) for label, should be list or numpy array

In [50]:
class_id_emb_attr.head()

,class_name,emb,attr
class_id,,,
ZJL1,goldfish,"[-0.036463, 0.34304, -0.32295, 0.36496, 0.5080...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0, 0.0, ..."
ZJL10,tarantula,"[-0.1564, 0.38850999999999997, -0.331580000000...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, ..."
ZJL100,drumstick,"[-0.15132, 0.18007, 0.12005, 0.21443, -0.40094...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
ZJL101,dumbbell,"[-0.3159, 0.15689, -0.037299, -0.2394800000000...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.1, 0.3, ..."
ZJL102,flagpole,"[0.06352200000000001, -0.17278, 0.12012, -0.15...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ..."


In [ ]:
 
data_test = open("D:\\data\\pic\\DatasetA_test_20180813\\DatasetA_test\\image.txt")
data_test = data_test.readlines()
path_test = r'D:\data\pic\DatasetA_test_20180813\DatasetA_test\test\\'
length = len(data_test)
data_test_x = np.zeros((length, 64, 64, 3))
for i in range(length):
    m = data_test[i][:-1]
    img = image.load_img(path_test + m)
    data_test_x[i] = image.img_to_array(img)
 
res = model.predict(data_test_x)
 
#欧氏距离：
data_atten_res = []
for i in range(160,241):
    key = 'ZJL' + str(i)
    data_atten_res.append(np.hstack((data_atten.loc[key][1:8], data_atten.loc[key][16:20], data_atten.loc[key][20:26])))
 
vec_result = []#np.zores((len(res[0]), 18))
for i in range(len(res[0])):
    vec_result.append(np.hstack((res[0][i], res[1][i], res[2][i])))
dis_list = []
 
for j in range(len(data_atten_res)):
    vec1 = vec_result[0]
    vec2 = data_atten_res[j]
    dist = np.linalg.norm(vec1 - vec2)
    dis_list.append(dist)
print("witch is : "+ str(dis_list.index(min(dis_list))) + " and min dis = " + str(min(dis_list)))